Start this first
```bash
$ litex_server uart /dev/ttyUSB0 1152000
```

In [8]:
%pylab notebook
from litex.soc.tools.remote import RemoteClient
from litescope.software.driver.analyzer import LiteScopeAnalyzerDriver
from struct import pack, unpack
import time
from hexdump import hexdump
from subprocess import run

def getPacket(dump, name="__main___mac_rx_cdc_source"):
    """
        dump a binary stream from litexscope capture
        rxn = name of data channel
        vn = name of data valid channel
        fn = name of output binary file
    """
    rxs = dump[name + "_payload_data"][::2]
    vs = dump[name + "_valid"][::2]
    return bytes([rx for v, rx in zip(vs, rxs) if v])

Populating the interactive namespace from numpy and matplotlib


# Connect to hardware

In [10]:
for i in range(32):
    try:
        r = RemoteClient(csr_csv="../build/csr.csv", debug=False, port=1234 + i)
        r.open()
        print("Connected to Port", 1234 + i)
        break
    except ConnectionRefusedError:
        pass
"".join([chr(r.read(r.bases.identifier_mem + i * 4) & 0xFF) for i in range(64)]).strip("\x00")

Connected to Port 1234


'LiteEth Base Design\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00LiteEth Base Design'

# Debug clocks

In [ ]:
r.regs.f_tx_value.read()

# Setup litescope

In [ ]:
analyzer = LiteScopeAnalyzerDriver(r.regs, "analyzer", config_csv="../build/analyzer.csv", debug=True)
analyzer.configure_subsampler(1)
# Trigger on transmitting a packet
analyzer.add_rising_edge_trigger("mac_tx_cdc_sink_valid")

In [ ]:
analyzer.run(offset=128, length=256)
analyzer.wait_done()
dump = analyzer.upload()
analyzer.save("dump.vcd")
analyzer.save("dump.py")

Run this to trigger it
```bash
$ ping 192.168.1.50
```

In [ ]:
!gtkwave dump.vcd

# Extract packet data as seen by the FPGA

In [ ]:
from dump import dump
with open("dump.hex", "w") as f:
    f.write(hexdump(getPacket(dump, "mac_rx_cdc_source"), "return"))
    f.write("\n")
    f.write(hexdump(getPacket(dump, "mac_tx_cdc_sink"), "return"))
!cat dump.hex
!text2pcap dump.hex dump.pcap
!rm -rf dump.hex

In [ ]:
!wireshark dump.pcap

In [ ]:
r.close()